In [ ]:
import numpy as np
import math
import pandas as pd
import tensorflow as tf

In [ ]:
#generate injection wellbore temperature profile
r1=0.1
ke=2
cf=4100
kf=0.5
alpha=1.0e-6 
df=1000 
a=0.05
b=21
Tinj=10
delta_t=36000
R=200
width=0.1
L=900
Num_train=1
m=900
NN=20
#logarithmic cylindrical coordinates:
r=np.zeros(NN+1)
delta_r=np.zeros(NN+1)
delta_rr=1/NN*math.log(R/r1)
for n in range(1,NN+2):
  r[n-1]=r1*math.pow(R/r1,(n-1)/NN)
  delta_r[n-1]=delta_rr*r[n-1]
T_injection_train=np.zeros((Num_train,m+1,NN+2))
T_heatup_train=np.zeros((Num_train,m+1,NN+2))
y_original=np.zeros((Num_train,m+1))
Z_train=np.zeros((Num_train,m+1))

Fra_p=np.zeros((Num_train,m+1))
L_p=np.zeros((Num_train,m+1))
Fra_flowrate=np.zeros((Num_train,m+1))
WW=np.zeros((Num_train))
for tt in range(0,Num_train):#number of training data
  W=np.random.uniform(1,9,size=1)#
  WW[tt]=W
  fra_num=2#fracture numbers along the wellbore
  flowrate=np.zeros(fra_num)
  flowrate[0]=np.random.uniform(0.75,0.91,size=1) 
  flowrate[1]=0.95-flowrate[0]
  L1=np.random.randint(150,900,size=fra_num) 
  L1=sorted(L1)
  while L1[1]-L1[0]<150:
    L1=np.random.randint(150,900,size=fra_num)
  oo=0
  for qq in L1:
    L_p[tt,int(qq)]=1
    Fra_flowrate[tt,int(qq)]=flowrate[oo]*W
    oo=oo+1
  Fra=np.zeros(fra_num)
  Fra_int=np.zeros(fra_num)
  for i in range(0,fra_num):
    Fra[i]=int(m*L1[i]/L)
    Fra_int[i]=int(Fra[i])
  for qq in Fra_int:
    Fra_p[tt,int(qq)]=1

  z=np.zeros(m+1)
  delta_z=np.zeros(m+1)
  for i in range(0,fra_num-1):
    for j in range(int(Fra[i]+1),int(Fra[i+1]-1)):
      if ((Fra[i+1]-1)-(Fra[i]+1))%2==0:
        N=int(((Fra[i+1]-1)-(Fra[i]+1))/2) 
        z1=np.zeros(N+1)
        delta_z1=np.zeros(N)
        for k in range(0,N+1):
          z1[k]=1/(1/(L1[i]+width/2)-k/N*(1/(L1[i]+width/2)-1/((L1[i]+L1[i+1])/2)))
        for k in range(0,N):
          delta_z1[k]=z1[k+1]-z1[k]
      elif ((Fra[i+1]-1)-(Fra[i]+1)+1)%2==0:
        N=int(((Fra[i+1]-1)-(Fra[i]+1)+1)/2)
        z1=np.zeros(N+1)
        delta_z1=np.zeros(N)
        L3=((L1[i+1]-width/2)-(2*N-2)*(L1[i]+width/2)+(4*(L1[i]+width/2)**2*(N**2-N)+(L1[i+1]-width/2)**2)**0.5)/2
        for k in range(0,N+1):
          z1[k]=1/(1/(L1[i]+width/2)-k/N*(1/(L1[i]+width/2)-1/L3))
        for k in range(0,N):
          delta_z1[k]=z1[k+1]-z1[k]
      delta_z12=np.zeros(int(Fra[i+1]-1-(Fra[i]+1)))
      for n in range(0,int(Fra[i+1]-1-(Fra[i]+1))):
        if n<N:
          delta_z12[n]=delta_z1[n]
        elif n>=N:
          delta_z12[n]=delta_z1[int(Fra[i+1]-1-(Fra[i]+1)-n-1)]
      delta_z[j]=delta_z12[int(j-(Fra[i]+1))]
    delta_z[int(Fra[i+1]-1)]=width/2
    delta_z[int(Fra[i])]=width/2
  z1=np.zeros(int(Fra[0]))
  delta_z1=np.zeros(int(Fra[0]-1))
  for k in range(0,int(Fra[0])):
    z1[k]=width/2*math.pow(L1[0]/(width/2),k/(Fra[0]-1))
  for k in range(0,int(Fra[0]-1)):
    delta_z1[k]=z1[k+1]-z1[k]
  for j in range(0,int(Fra[0]-1)):
    delta_z[j]=delta_z1[int(Fra[0]-2-j)]
  delta_z[int(Fra[0]-1)]=width/2
  z2=np.zeros(int(m-Fra[fra_num-1]))
  delta_z2=np.zeros(int(m-Fra[fra_num-1]-1))
  for k in range(0,int(m-Fra[fra_num-1])):
    z2[k]=width/2*math.pow((L-L1[fra_num-1])/(width/2),k/(m-Fra[fra_num-1]-1))
  for k in range(0,int(m-Fra[fra_num-1]-1)):
    delta_z2[k]=z2[k+1]-z2[k]
  for j in range(int(Fra[fra_num-1]+1),m):
    delta_z[j]=delta_z2[int(j-Fra[fra_num-1]-1)]
  delta_z[int(Fra[fra_num-1])]=width/2
  delta_z[m]=delta_z[m-1]
  z[0]=0
  for j in range(1,m+1):
    z[j]=sum(delta_z[0:j])

  #wellbore velocity&fracture velocity
  U=np.zeros(m+1)
  V=np.zeros((m+1,NN+2))
  for i in range(0,fra_num):
    V[int(Fra[i]),0]=W*flowrate[i]/df/(width)/(2*math.pi*r1)#每个fracture流出流量
    V[:,1]=V[:,0]
  for n in range(2,NN+2):
    V[:,n]=V[:,1]*r1/r[n-1]
  U[0]=W/math.pi/r1/r1/df
  for j in range(0,m):
    U[j+1]=U[j]-2*width/r1*V[j,0]
  # initial condition
  T=np.zeros((m+1,NN+2))
  T_injection=np.zeros((m+1,NN+2))
  T_new_f=np.zeros((m+1,NN+2))
  y=np.zeros(m*(NN+2))
  M=np.zeros((m*(NN+2),m*(NN+2)))
  for p in range(0,m+1):
    T[p,0:NN+2]=a*z[p]+b

  # boundary condition
    T[0,0]=Tinj 
    T[0,1:]=b 
    #fracture Nu=0.332Rex^0.5Pr^(1/3)
    h=10#math.pow(df*V[int(Fra[0]),0]*r1/1.01*1000,0.5)*math.pow(6.9,1/3)*0.332*kf/r1
    Uf=10#math.pow(df*U[0]*r1/1.01*1000,0.5)*math.pow(6.9,1/3)*0.332*kf/r1
  # wellbore
  for t in range(0,3,1):#1days injection###########################################一会时间别忘了改回来
      for j in range(1,2):
            D=cf/delta_t-cf*(-U[j])/delta_z[j-1]+2/r1/df*Uf
            M[0,0]=D
            M[0,m]=-2/r1/df*Uf
            y[0]=cf/delta_t*T[j,0]-cf*(-U[j])/delta_z[j-1]*T[0,0]-4*(-U[j])*(-U[j])/r1*V[j,0]
      for j in range(2,m+1):
        if j in Fra[:]:
          D=cf/delta_t+cf*U[j]/(width/2)+2*kf/df/(width/2)/(width/2)
          M[j-1,j-1]=D
          M[j-1,j-2]=-cf*U[j]/(width/2)-kf/df/(width/2)/(width/2)
          M[j-1,j]=-kf/df/(width/2)/(width/2)
          y[j-1]=cf/delta_t*T[j,0]-4*U[j]*U[j]/r1*V[j,0]
        else:
          D=cf/delta_t+cf*U[j]/delta_z[j-1]+2/r1/df*Uf+2*kf/df/(delta_z[j]+delta_z[j-1])*(1/delta_z[j]+1/delta_z[j-1])
          M[j-1,j-1]=D
          #M[j-1,j-2]=-cf*U[j]/delta_z[j-1]
          M[j-1,j]=-2*kf/df/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
          M[j-1,j-2]=-2*kf/df/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]-cf*U[j]/delta_z[j-1]
          M[j-1,j-1+m]=-2/r1/df*Uf
          y[j-1]=cf/delta_t*T[j,0]-4*U[j]*U[j]/r1*V[j,0]    
  #formation
      #n=1
      for n in range(1,2):
        for j in range(1,2):
          B=1/delta_t+2*alpha/delta_z[j]/delta_z[j-1]+alpha/delta_r[1-1]/delta_r[1-1]+Uf/ke*alpha/delta_r[1-1]
          M[0+(n)*m,0+(n)*m]=B
          M[0+(n)*m,1+(n)*m]=-(2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j])
          M[0+m,m+m]=-(alpha/delta_r[1-1]/delta_r[1-1])
          M[0+m,m-m]=-Uf/ke*alpha/delta_r[1-1]
          y[0+(n)*m]=1/delta_t*T[1,n]+(2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j-1])*T[0,n]
        for j in range(2,m):
          if j in Fra[:]:
            M[j-1+m,j-1+m]=1
            M[j-1+m,j-1+m-m]=-1
            y[j-1+m]=0 
          elif j+1 in Fra[:]:#j=Fra-1
            B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j-1]+h/ke)+alpha/delta_r[1-1]/delta_r[1-1]+Uf/ke*alpha/delta_r[1-1]
            M[j-1+(n)*m,j-1+(n)*m]=B1
            M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]
            M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*h/ke
            M[j-1+m,j-1+m+m]=-(alpha/delta_r[1-1]/delta_r[1-1])
            M[j-1+m,j-1+m-m]=-Uf/ke*alpha/delta_r[1-1]
            y[j-1+(n)*m]=1/delta_t*T[j,n]

          elif j-1 in Fra[:-1]:#j==Fra+1
            B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j]+h/ke)+alpha/delta_r[1-1]/delta_r[1-1]+Uf/ke*alpha/delta_r[1-1]
            M[j-1+(n)*m,j-1+(n)*m]=B1
            M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
            M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*h/ke
            M[j-1+m,j-1+m+m]=-(alpha/delta_r[1-1]/delta_r[1-1])
            M[j-1+m,j-1+m-m]=-Uf/ke*alpha/delta_r[1-1]
            y[j-1+(n)*m]=1/delta_t*T[j,n]
          elif j-1 == Fra[-1]:#j==Fra+1
            B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j]+h/ke)+alpha/delta_r[1-1]/delta_r[1-1]+kf/ke*alpha/delta_r[1-1]/r1
            M[j-1+(n)*m,j-1+(n)*m]=B1
            M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
            M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*h/ke
            M[j-1+m,j-1+m+m]=-(alpha/delta_r[1-1]/delta_r[1-1])
            M[j-1+m,j-1+m-m]=-kf/ke*alpha/delta_r[1-1]/r1
            y[j-1+(n)*m]=1/delta_t*T[j,n]  
            
          elif j>Fra[-1]+1: 
            B=1/delta_t+2*alpha/delta_z[j]/delta_z[j-1]+alpha/delta_r[1-1]/delta_r[1-1]+kf/ke*alpha/delta_r[1-1]/r1#+Uf/ke*alpha/delta_r[1-1]
            M[j-1+(n)*m,j-1+(n)*m]=B
            M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j-1]
            M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j]
            M[j-1+m,j-1+m+m]=-(alpha/delta_r[1-1]/delta_r[1-1])
            M[j-1+m,j-1+m-m]=-kf/ke*alpha/delta_r[1-1]/r1#-Uf/ke*alpha/delta_r[1-1]
            y[j-1+(n)*m]=1/delta_t*T[j,n]
          else:
            B=1/delta_t+2*alpha/delta_z[j]/delta_z[j-1]+alpha/delta_r[1-1]/delta_r[1-1]+Uf/ke*alpha/delta_r[1-1]
            M[j-1+(n)*m,j-1+(n)*m]=B
            M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j-1]
            M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j]
            M[j-1+m,j-1+m+m]=-(alpha/delta_r[1-1]/delta_r[1-1])
            M[j-1+m,j-1+m-m]=-Uf/ke*alpha/delta_r[1-1]
            y[j-1+(n)*m]=1/delta_t*T[j,n]
        #j=m
        B=1/delta_t+2*alpha/delta_z[m]/delta_z[m-1]+alpha/delta_r[1-1]/delta_r[1-1]+Uf/ke*alpha/delta_r[1-1]
        M[m-1+(n)*m,m-1+(n)*m]=B
        M[m-1+(n)*m,m-2+(n)*m]=-2*alpha/(delta_z[m]+delta_z[m-1])/delta_z[m-1]
        M[m-1+m,m-1+m+m]=-(alpha/delta_r[1-1]/delta_r[1-1])
        M[m-1+m,m-1+m-m]=-Uf/ke*alpha/delta_r[1-1]
        y[m-1+(n)*m]=1/delta_t*T[m,n]+2*alpha/(delta_z[m]+delta_z[m-1])/delta_z[m]*(a*L+b)
  #n=2...N
      for n in range(2,NN+1):
        for j in range(1,2):
          B=1/delta_t+2*alpha/delta_z[j]/delta_z[j-1]+2*alpha/delta_r[n-1]/delta_r[n-1]
          M[0+(n)*m,0+(n)*m]=B
          M[0+(n)*m,1+(n)*m]=-(2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j])
          M[0+(n)*m,m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
          M[0+(n)*m,-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
          y[0+(n)*m]=1/delta_t*T[1,n]+(2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j-1])*T[0,n]

        for j in range(2,m):
          if j in Fra[:]:
            A1=cf/delta_t-(-cf*V[j,n])/delta_r[n-1]+2*kf/df/delta_r[n-1]/delta_r[n-1]+h/df/width+h/df/width
            M[j-1+(n)*m,j-1+(n)*m]=A1
            M[j-1+(n)*m,j+(n)*m]=-h/df/width
            M[j-1+(n)*m,j-2+(n)*m]=-h/df/width
            M[j-1+(n)*m,j-1+(n)*m-m]=(-cf*V[j,n])/delta_r[n-1]-kf/df/delta_r[n-1]/delta_r[n-1]
            M[j-1+(n)*m,j-1+(n)*m+m]=-kf/df/delta_r[n-1]/delta_r[n-1]
            y[j-1+(n)*m]=cf/delta_t*T[j,n]

          elif j+1 in Fra[:]:#j==Fra-1
            B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j-1]+h/ke)+2*alpha/delta_r[n-1]/delta_r[n-1]
            M[j-1+(n)*m,j-1+(n)*m]=B1
            M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]
            M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*h/ke
            M[j-1+(n)*m,j-1+m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
            M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
            y[j-1+(n)*m]=1/delta_t*T[j,n]

          elif j-1 in Fra[:]:#j==Fra+1
            B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j]+h/ke)+2*alpha/delta_r[n-1]/delta_r[n-1]
            M[j-1+(n)*m,j-1+(n)*m]=B1
            M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
            M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*h/ke
            M[j-1+(n)*m,j-1+m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
            M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
            y[j-1+(n)*m]=1/delta_t*T[j,n]
          else:#j>Fra+1 或者j<Fra-1
              B=1/delta_t+2*alpha/delta_z[j]/delta_z[j-1]+2*alpha/delta_r[n-1]/delta_r[n-1]
              M[j-1+(n)*m,j-1+(n)*m]=B
              M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j-1]
              M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j]
              M[j-1+(n)*m,j-1+m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
              M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
              y[j-1+(n)*m]=1/delta_t*T[j,n]
        #j=m
        B=1/delta_t+2*alpha/delta_z[m]/delta_z[m-1]+2*alpha/delta_r[n-1]/delta_r[n-1]
        M[m-1+(n)*m,m-1+(n)*m]=B
        M[m-1+(n)*m,m-2+(n)*m]=-2*alpha/(delta_z[m]+delta_z[m-1])/delta_z[m-1]
        M[m-1+(n)*m,m-1+m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
        M[m-1+(n)*m,m-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
        y[m-1+(n)*m]=1/delta_t*T[m,n]+2*alpha/(delta_z[m]+delta_z[m-1])/delta_z[m]*(a*L+b)

  #n=NN+1
      for n in range(NN+1,NN+2):
        for j in range(1,2):
          k=Fra-1-j
          B=1/delta_t+2*alpha/delta_z[j]/delta_z[j-1]+2*alpha/delta_r[n-1]/delta_r[n-1]
          M[0+(n)*m,0+(n)*m]=B
          M[0+(n)*m,1+(n)*m]=-(2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j])
          M[0+(n)*m,-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
          y[0+(n)*m]=1/delta_t*T[1,n]+(2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j-1])*T[0,n]+alpha/delta_r[n-1]/delta_r[n-1]*(a*z[j]+b)

        for j in range(2,m):
          if j in Fra[:]:
            A1=cf/delta_t-(-cf*V[j,n])/delta_r[n-1]+2*kf/df/delta_r[n-1]/delta_r[n-1]+h/df/width+h/df/width
            M[j-1+(n)*m,j-1+(n)*m]=A1
            M[j-1+(n)*m,j+(n)*m]=-h/df/width
            M[j-1+(n)*m,j-2+(n)*m]=-h/df/width
            M[j-1+(n)*m,j-1+(n)*m-m]=(-cf*V[j,n])/delta_r[n-1]-kf/df/delta_r[n-1]/delta_r[n-1]
            y[j-1+(n)*m]=cf/delta_t*T[j,n]+kf/df/delta_r[n-1]/delta_r[n-1]*(a*z[j]+b)

          elif j+1 in Fra[:]:#j==Fra-1
            B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j-1]+h/ke)+2*alpha/delta_r[n-1]/delta_r[n-1]
            M[j-1+(n)*m,j-1+(n)*m]=B1
            M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]
            M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*h/ke
            M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
            y[j-1+(n)*m]=1/delta_t*T[j,n]+alpha/delta_r[n-1]/delta_r[n-1]*(a*z[j]+b)

          elif j-1 in Fra[:]:#j==Fra+1
            B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j]+h/ke)+2*alpha/delta_r[n-1]/delta_r[n-1]
            M[j-1+(n)*m,j-1+(n)*m]=B1
            M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
            M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*h/ke
            M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
            y[j-1+(n)*m]=1/delta_t*T[j,n]+alpha/delta_r[n-1]/delta_r[n-1]*(a*z[j]+b)
          else: #j<Fra-1 or j>Fra+1
              B=1/delta_t+2*alpha/delta_z[j]/delta_z[j-1]+2*alpha/delta_r[n-1]/delta_r[n-1]
              M[j-1+(n)*m,j-1+(n)*m]=B
              M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]
              M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
              M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
              y[j-1+(n)*m]=1/delta_t*T[j,n]+alpha/delta_r[n-1]/delta_r[n-1]*(a*z[j]+b)
        #j=m
        B=1/delta_t+2*alpha/delta_z[m]/delta_z[m-1]+2*alpha/delta_r[n-1]/delta_r[n-1]
        M[m-1+(n)*m,m-1+(n)*m]=B
        M[m-1+(n)*m,m-2+(n)*m]=-2*alpha/(delta_z[m]+delta_z[m-1])*1/delta_z[m-1]
        M[m-1+(n)*m,m-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
        y[m-1+(n)*m]=1/delta_t*T[m,n]+2*alpha/(delta_z[m]+delta_z[m-1])*1/delta_z[m]*(a*L+b)+alpha/delta_r[n-1]/delta_r[n-1]*(a*z[j]+b)
      #solve
      y_tf=np.expand_dims(y,axis=1)
      M_gpu=tf.convert_to_tensor(M)
      y_gpu=tf.convert_to_tensor(y_tf)
      with tf.device('/GPU:0'):
          T_new_f=tf.linalg.solve(M_gpu,y_gpu)
      T_new_f.numpy()
      for n in range(0,NN+2):
            for j in range(1,m+1):
              k=j+(n)*m-1
              T[j,n]=T_new_f[k]#formation更新温度
  for k in range(0,m):
    if Fra_p[tt,k]==1:
      dd=k
      y_original[tt,dd]=1##这其实y_original和Fra_p 是一样的吧...
  T_injection_train[tt,:,:]=T
  Z_train[tt,:]=z
  print(f'tt={tt}')

df3=pd.DataFrame(T_injection_train.reshape(Num_train*(m+1),NN+2))
df3.to_csv('T_injection_train_multifrac_iso.csv',index=None)

df2=pd.DataFrame(y_original)
df2.to_csv('position_heatup_train_multifrac_iso.csv',index=None)

df4=pd.DataFrame(Z_train)
df4.to_csv('z_multi_fracture_multifrac_iso.csv',index=None)

df5=pd.DataFrame(L_p)
df5.to_csv('fracture_L1_multifrac_iso.csv',index=None)

df6=pd.DataFrame(Fra_flowrate)
df6.to_csv('fracflowrate_heatup_train_multifrac_iso.csv',index=None)


tt=0
tt=1
tt=2
tt=3
tt=4
tt=5
tt=6
tt=7
tt=8
tt=9
tt=10
tt=11
tt=12
tt=13
tt=14
tt=15
tt=16
tt=17
tt=18
tt=19
tt=20
tt=21
tt=22
tt=23
tt=24
tt=25
tt=26
tt=27
tt=28
tt=29
tt=30
tt=31
tt=32
tt=33
tt=34
tt=35
tt=36
tt=37
tt=38
tt=39
tt=40
tt=41
tt=42
tt=43
tt=44
tt=45
tt=46
tt=47
tt=48
tt=49
tt=50
tt=51
tt=52
tt=53
tt=54
tt=55


/share/software/user/open/py-jupyter/1.0.0_py36/lib/python3.6/site-packages/ipykernel_launcher.py:105: RuntimeWarning: invalid value encountered in double_scalars


tt=56
tt=57
tt=58
tt=59
tt=60
tt=61
tt=62
tt=63
tt=64
tt=65
tt=66
tt=67
tt=68
tt=69
tt=70
tt=71
tt=72
tt=73
tt=74
tt=75
tt=76
tt=77
tt=78
tt=79
tt=80
tt=81
tt=82
tt=83
tt=84
tt=85
tt=86
tt=87
tt=88
tt=89
tt=90
tt=91
tt=92
tt=93
tt=94
tt=95
tt=96
tt=97
tt=98
tt=99
tt=100
tt=101
tt=102
tt=103
tt=104
tt=105
tt=106
tt=107
tt=108
tt=109
tt=110
tt=111
tt=112
tt=113
tt=114
tt=115
tt=116
tt=117
tt=118
tt=119
tt=120
tt=121
tt=122
tt=123
tt=124
tt=125
tt=126
tt=127
tt=128
tt=129
tt=130
tt=131
tt=132
tt=133
tt=134
tt=135
tt=136
tt=137
tt=138
tt=139
tt=140
tt=141
tt=142
tt=143
tt=144
tt=145
tt=146
tt=147
tt=148
tt=149
tt=150
tt=151
tt=152
tt=153
tt=154
tt=155
tt=156
tt=157
tt=158
tt=159
tt=160
tt=161
tt=162
tt=163
tt=164
tt=165
tt=166
tt=167
tt=168
tt=169
tt=170
tt=171
tt=172
tt=173
tt=174
tt=175
tt=176
tt=177
tt=178
tt=179
tt=180
tt=181
tt=182
tt=183
tt=184
tt=185
tt=186
tt=187
tt=188
tt=189
tt=190
tt=191
tt=192
tt=193
tt=194
tt=195
tt=196
tt=197
tt=198
tt=199
tt=200
tt=201
tt=202
tt=203
tt=204
t

In [ ]:
#generate static stage wellbore temperature 
#use geotehrmal gradient as initial field for static isothermal internal flow
r1=0.1
ke=2
cf=4100
kf=0.5
alpha=1.0e-6 
df=1000
a=0.2
b=21
Tinj=10
R=200 
width=0.1
L=900
m=900
NN=20 
delta_t=3600*10
heat_up_number=1#single static stage
Num_train=800
WW=2
L_p=np.array(pd.read_csv('position_heatup_train_multifrac_iso.csv'))#load the same fracture positions with that generated during injection for each case
Fra_flowrate=np.array(pd.read_csv('fracflowrate_heatup_train_multifrac_iso.csv'))#load the fracture flow rate during injection for each case

T_heatup_output=np.zeros((Num_train,m+1,heat_up_number))
Z_train_out=np.zeros((Num_train,m+1,heat_up_number))
T_heatup_train=np.zeros((Num_train,m+1,NN+2))
T_frac_r=np.zeros((heat_up_number,NN+2))
Fra_p=np.zeros((Num_train,m+1))
Z_train=np.zeros((Num_train,m+1))


r=np.zeros(NN+1)
delta_r=np.zeros(NN+1)
delta_rr=1/NN*math.log(R/r1)
for n in range(1,NN+2):
    r[n-1]=r1*math.pow(R/r1,(n-1)/NN)
    delta_r[n-1]=delta_rr*r[n-1]
    
for tt in range(0,Num_train):
    W=2
    L1=[]
    for k in range(0,m):
        if L_p[tt,k]==1:
            L1.append(k)
    fra_num=len(L1)
    Fra=np.zeros(fra_num)
    flowrate=np.zeros(fra_num)
    for i in range(0,fra_num):
        Fra[i]=int(m*L1[i]/L)
        flowrate[i]=Fra_flowrate[tt,int(Fra[i])]
    T_hu=np.zeros((m+1,NN+2))

    flowrate[fra_num-1]=4
    UU=np.zeros(m+1)
    VV=np.zeros((m+1,NN+2))
    VV[int(Fra[fra_num-2]),0]=W*flowrate[fra_num-1]/df/(width)/(2*math.pi*r1)#
    VV[int(Fra[fra_num-1]),0]=-W*flowrate[fra_num-1]/df/(width)/(2*math.pi*r1)
    VV[:,1]=VV[:,0]

    for n in range(2,NN+2):
      VV[:,n]=VV[:,1]*r1/r[n-1]
    for j in range(1,m+1):
      UU[j]=UU[j-1]-2*width/r1*VV[j,0]

    h=10#math.pow(df*VV[int(Fra[fra_num-2]),0]*r1/1.01*1000,0.5)*math.pow(6.9,1/3)*0.332*kf/r1
    Uf=10
    z=np.zeros(m+1)
    delta_z=np.zeros(m+1)
    for i in range(0,fra_num-1):
        for j in range(int(Fra[i]+1),int(Fra[i+1]-1)):
            if ((Fra[i+1]-1)-(Fra[i]+1))%2==0:
                N=int(((Fra[i+1]-1)-(Fra[i]+1))/2) 
                z1=np.zeros(N+1)
                delta_z1=np.zeros(N)
                for k in range(0,N+1):
                    z1[k]=1/(1/(L1[i]+width/2)-k/N*(1/(L1[i]+width/2)-1/((L1[i]+L1[i+1])/2)))
                for k in range(0,N):
                    delta_z1[k]=z1[k+1]-z1[k]
            elif ((Fra[i+1]-1)-(Fra[i]+1)+1)%2==0:
                N=int(((Fra[i+1]-1)-(Fra[i]+1)+1)/2)
                z1=np.zeros(N+1)
                delta_z1=np.zeros(N)
                L3=((L1[i+1]-width/2)-(2*N-2)*(L1[i]+width/2)+(4*(L1[i]+width/2)**2*(N**2-N)+(L1[i+1]-width/2)**2)**0.5)/2
                for k in range(0,N+1):
                    z1[k]=1/(1/(L1[i]+width/2)-k/N*(1/(L1[i]+width/2)-1/L3))
                for k in range(0,N):
                    delta_z1[k]=z1[k+1]-z1[k]
            delta_z12=np.zeros(int(Fra[i+1]-1-(Fra[i]+1)))
            for n in range(0,int(Fra[i+1]-1-(Fra[i]+1))):
                if n<N:
                    delta_z12[n]=delta_z1[n]
                elif n>=N:
                    delta_z12[n]=delta_z1[int(Fra[i+1]-1-(Fra[i]+1)-n-1)]
            delta_z[j]=delta_z12[int(j-(Fra[i]+1))]
        delta_z[int(Fra[i+1]-1)]=width/2
        delta_z[int(Fra[i])]=width/2
    z1=np.zeros(int(Fra[0]))
    delta_z1=np.zeros(int(Fra[0]-1))
    for k in range(0,int(Fra[0])):
        z1[k]=width/2*math.pow(L1[0]/(width/2),k/(Fra[0]-1))
    for k in range(0,int(Fra[0]-1)):
        delta_z1[k]=z1[k+1]-z1[k]
    for j in range(0,int(Fra[0]-1)):
        delta_z[j]=delta_z1[int(Fra[0]-2-j)]
    delta_z[int(Fra[0]-1)]=width/2
    z2=np.zeros(int(m-Fra[fra_num-1]))
    delta_z2=np.zeros(int(m-Fra[fra_num-1]-1))
    for k in range(0,int(m-Fra[fra_num-1])):
        z2[k]=width/2*math.pow((L-L1[fra_num-1])/(width/2),k/(m-Fra[fra_num-1]-1))
    for k in range(0,int(m-Fra[fra_num-1]-1)):
        delta_z2[k]=z2[k+1]-z2[k]
    for j in range(int(Fra[fra_num-1]+1),m):
        delta_z[j]=delta_z2[int(j-Fra[fra_num-1]-1)]
    delta_z[int(Fra[fra_num-1])]=width/2
    delta_z[m]=delta_z[m-1]
    z[0]=0
    for j in range(1,m+1):
        z[j]=sum(delta_z[0:j])
      # initial condition
    T_new_f=np.zeros((m+1,NN+2))
    y=np.zeros(m*(NN+2))
    M=np.zeros((m*(NN+2),m*(NN+2)))
    for p in range(0,m+1):#use geothermal gradient as initial field for static internal flow
        T_hu[p,0:NN+2]=a*z[p]+b
      # boundary condition
    T_hu[0,0]=b #wellbore inlet 
    T_hu[0,1:]=b #formation upper boundary


    for ii in range(0,heat_up_number):
        #wellbore
        for t in range(0,1,1):
            for j in range(1,2):
                D=cf/delta_t+2/r1/df*kf/r1+2*kf/df/(delta_z[j]+delta_z[j-1])*(1/delta_z[j-1]+1/delta_z[j])
                M[0,0]=D
                M[0,1]=-2*kf/df/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
                M[0,m]=-2/r1/df*kf/r1
                y[0]=cf/delta_t*T_hu[j,0]+2*kf/df/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]*T_hu[0,0]
            for j in range(2,m):
                if j in range(int(Fra[fra_num-2]),int(Fra[fra_num-1])):
                  if j in Fra:
                    D=cf/delta_t-cf*UU[j]/(width/2)+2*kf/df/(width/2)/(width/2)
                    M[j-1,j-1]=D
                    M[j-1,j-2]=-kf/df/(width/2)/(width/2)
                    M[j-1,j]=cf*UU[j]/(width/2)-kf/df/(width/2)/(width/2)
                    y[j-1]=cf/delta_t*T_hu[j,0]-4*UU[j]*UU[j]/r1*VV[j,0]
                  else:
                    D=cf/delta_t-cf*UU[j]/delta_z[j]+2/r1/df*Uf+2*kf/df/(delta_z[j]+delta_z[j-1])*(1/delta_z[j-1]+1/delta_z[j])
                    M[j-1,j-1]=D
                    M[j-1,j]=cf*UU[j]/delta_z[j]-2*kf/df/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
                    M[j-1,j-2]=-2*kf/df/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]
                    M[j-1,j-1+m]=-2/r1/df*Uf
                    y[j-1]=cf/delta_t*T_hu[j,0]-4*UU[j]*UU[j]/r1*VV[j,0]
                elif j==int(Fra[fra_num-1]):
                    D=cf/delta_t-cf*UU[j]/(width/2)+2*kf/df/(width/2)/(width/2)-2*VV[j,0]*cf/r1+2/r1*kf/r1/df
                    M[j-1,j-1]=D
                    M[j-1,j-2]=-kf/df/(width/2)/(width/2)
                    M[j-1,j]=cf*UU[j]/(width/2)-kf/df/(width/2)/(width/2)
                    M[j-1,j-1+m]=2*VV[j,0]*cf/r1-2/r1*kf/r1/df
                    y[j-1]=cf/delta_t*T_hu[j,0]-4*UU[j]*UU[j]/r1*VV[j,0]
                else:
                  if j in Fra:
                    D=cf/delta_t+2*kf/df/(width/2)/(width/2)
                    M[j-1,j-1]=D
                    M[j-1,j-2]=-kf/df/(width/2)/(width/2)
                    M[j-1,j]=-kf/df/(width/2)/(width/2)
                    M[j-1,j-1+m]=0
                    y[j-1]=cf/delta_t*T_hu[j,0]
                  else: #j!=Fra
                    D=cf/delta_t+2/r1/df*kf/r1+2*kf/df/(delta_z[j]+delta_z[j-1])*(1/delta_z[j-1]+1/delta_z[j])
                    M[j-1,j-1]=D
                    M[j-1,j]=-2*kf/df/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
                    M[j-1,j-2]=-2*kf/df/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]
                    M[j-1,j-1+m]=-2/r1/df*kf/r1
                    y[j-1]=cf/delta_t*T_hu[j,0]
            D=cf/delta_t+2/r1/df*kf/r1+2*kf/df/(delta_z[m]+delta_z[m-1])*(1/delta_z[m-1]+1/delta_z[m])
            M[m-1,m-1]=D
            M[m-1,m-2]=-2*kf/df/(delta_z[m]+delta_z[m-1])*1/delta_z[m-1]
            M[m-1,m-1+m]=-2/r1/df*kf/r1
            y[m-1]=cf/delta_t*T_hu[j,0]+2*kf/df/(delta_z[m]+delta_z[m-1])*1/delta_z[m]*(a*L+b)
            #formation
            #n=1
            for n in range(1,2):
                for j in range(1,2):
                    B=1/delta_t+2*alpha/delta_z[j]/delta_z[j-1]+alpha/delta_r[1-1]/delta_r[1-1]+kf/ke*alpha/delta_r[1-1]/r1
                    M[0+(n)*m,0+(n)*m]=B
                    M[0+(n)*m,1+(n)*m]=-(2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j])
                    M[0+m,m+m]=-(alpha/delta_r[1-1]/delta_r[1-1])
                    M[0+m,m-m]=-kf/ke*alpha/delta_r[1-1]/r1
                    y[0+(n)*m]=1/delta_t*T_hu[1,n]+(2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j-1])*T_hu[0,n]
                for j in range(2,m):
                    if j in Fra[:-1]:
                        M[j-1+m,j-1+m]=1
                        M[j-1+m,j-1+m-m]=-1
                        y[j-1+m]=0    
                    elif j ==int(Fra[fra_num-1]):
                        A1=cf/delta_t-(cf*VV[j,n])/delta_r[n-1]+kf/df/delta_r[n-1]/delta_r[n-1]+h/df/width+h/df/width+kf/df/delta_r[n-1]/r1

                        M[j-1+(n)*m,j-1+(n)*m]=A1
                        M[j-1+(n)*m,j+(n)*m]=-h/df/width
                        M[j-1+(n)*m,j-2+(n)*m]=-h/df/width
                        M[j-1+(n)*m,j-1+(n)*m-m]=-kf/df/delta_r[n-1]/r1
                        M[j-1+(n)*m,j-1+(n)*m+m]=-kf/df/delta_r[n-1]/delta_r[n-1]+(cf*VV[j,n])/delta_r[n-1]
                        y[j-1+(n)*m]=cf/delta_t*T_hu[j,n]    
                    elif j+1 in Fra[:fra_num-2]:#j==Fra-1
                        B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j-1]+kf/ke*1/delta_z[j])+alpha/delta_r[1-1]/delta_r[1-1]+kf/ke*alpha/delta_r[1-1]/r1
                        M[j-1+(n)*m,j-1+(n)*m]=B1
                        M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]
                        M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*kf/ke*1/delta_z[j]
                        M[j-1+m,j-1+m+m]=-(alpha/delta_r[1-1]/delta_r[1-1])
                        M[j-1+m,j-1+m-m]=-kf/ke*alpha/delta_r[1-1]/r1
                        y[j-1+(n)*m]=1/delta_t*T_hu[j,n]

                    elif j-1 in Fra[:fra_num-2]:#j==Fra+1
                        B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j]+kf/ke*1/delta_z[j-1])+alpha/delta_r[1-1]/delta_r[1-1]+kf/ke*alpha/delta_r[1-1]/r1
                        M[j-1+(n)*m,j-1+(n)*m]=B1
                        M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
                        M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*kf/ke*1/delta_z[j-1]
                        M[j-1+m,j-1+m+m]=-(alpha/delta_r[1-1]/delta_r[1-1])
                        M[j-1+m,j-1+m-m]=-kf/ke*alpha/delta_r[1-1]/r1
                        y[j-1+(n)*m]=1/delta_t*T_hu[j,n]
                    elif j+1 == Fra[fra_num-2]:#j==Fra-1
                        B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j-1]+h/ke)+alpha/delta_r[1-1]/delta_r[1-1]+kf/ke*alpha/delta_r[1-1]/r1
                        M[j-1+(n)*m,j-1+(n)*m]=B1
                        M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]
                        M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*h/ke
                        M[j-1+m,j-1+m+m]=-(alpha/delta_r[1-1]/delta_r[1-1])
                        M[j-1+m,j-1+m-m]=-kf/ke*alpha/delta_r[1-1]/r1
                        y[j-1+(n)*m]=1/delta_t*T_hu[j,n]   
                    elif j-1 == Fra[fra_num-1]:#j==Fra+1,last frac
                        B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j]+h/ke)+alpha/delta_r[1-1]/delta_r[1-1]+kf/ke*alpha/delta_r[1-1]/r1
                        M[j-1+(n)*m,j-1+(n)*m]=B1
                        M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
                        M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*h/ke
                        M[j-1+m,j-1+m+m]=-(alpha/delta_r[1-1]/delta_r[1-1])
                        M[j-1+m,j-1+m-m]=-kf/ke*alpha/delta_r[1-1]/r1
                        y[j-1+(n)*m]=1/delta_t*T_hu[j,n]
                    elif j in range(int(Fra[-2])+1,int(Fra[-1])):
                        if j-1 in Fra:#j==Fra+1
                          B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j]+h/ke)+alpha/delta_r[1-1]/delta_r[1-1]+Uf/ke*alpha/delta_r[1-1]
                          M[j-1+(n)*m,j-1+(n)*m]=B1
                          M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
                          M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*h/ke
                          M[j-1+m,j-1+m+m]=-(alpha/delta_r[1-1]/delta_r[1-1])
                          M[j-1+m,j-1+m-m]=-Uf/ke*alpha/delta_r[1-1]
                          y[j-1+(n)*m]=1/delta_t*T_hu[j,n]
                        else: 
                          B=1/delta_t+2*alpha/delta_z[j]/delta_z[j-1]+alpha/delta_r[1-1]/delta_r[1-1]+Uf/ke*alpha/delta_r[1-1]
                          M[j-1+(n)*m,j-1+(n)*m]=B
                          M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j-1]
                          M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j]
                          M[j-1+m,j-1+m+m]=-(alpha/delta_r[1-1]/delta_r[1-1])
                          M[j-1+m,j-1+m-m]=-Uf/ke*alpha/delta_r[1-1]
                          y[j-1+(n)*m]=1/delta_t*T_hu[j,n]
                    else:
                        B=1/delta_t+2*alpha/delta_z[j]/delta_z[j-1]+alpha/delta_r[1-1]/delta_r[1-1]+kf/ke*alpha/delta_r[1-1]/r1
                        M[j-1+(n)*m,j-1+(n)*m]=B
                        M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]
                        M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
                        M[j-1+m,j-1+m+m]=-(alpha/delta_r[1-1]/delta_r[1-1])
                        M[j-1+m,j-1+m-m]=-kf/ke*alpha/delta_r[1-1]/r1
                        y[j-1+(n)*m]=1/delta_t*T_hu[j,n]
              #j=m
                B=1/delta_t+2*alpha/delta_z[m]/delta_z[m-1]+alpha/delta_r[1-1]/delta_r[1-1]+kf/ke*alpha/delta_r[1-1]/r1
                M[m-1+(n)*m,m-1+(n)*m]=B
                M[m-1+(n)*m,m-2+(n)*m]=-2*alpha/(delta_z[m]+delta_z[m-1])*1/delta_z[m-1]
                M[m-1+m,m-1+m+m]=-(alpha/delta_r[1-1]/delta_r[1-1])
                M[m-1+m,m-1+m-m]=-kf/ke*alpha/delta_r[1-1]/r1
                y[m-1+(n)*m]=1/delta_t*T_hu[m,n]+2*alpha/(delta_z[m]+delta_z[m-1])*1/delta_z[m]*(a*L+b)
            #n=2...N
            for n in range(2,NN+1):
                for j in range(1,2):
                    B=1/delta_t+2*alpha/delta_z[j]/delta_z[j-1]+2*alpha/delta_r[n-1]/delta_r[n-1]
                    M[0+(n)*m,0+(n)*m]=B
                    M[0+(n)*m,1+(n)*m]=-(2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j])
                    M[0+(n)*m,m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                    M[0+(n)*m,-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                    y[0+(n)*m]=1/delta_t*T_hu[1,n]+(2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j-1])*T_hu[0,n]

                for j in range(2,m):
                    if j in Fra[:fra_num-2]:
                        A1=cf/delta_t+2*kf/df/delta_r[n-1]/delta_r[n-1]+kf/df/width/(width/2)+kf/df/width/(width/2)
                        M[j-1+(n)*m,j-1+(n)*m]=A1
                        M[j-1+(n)*m,j+(n)*m]=-kf/df/width/(width/2)
                        M[j-1+(n)*m,j-2+(n)*m]=-kf/df/width/(width/2)
                        M[j-1+(n)*m,j-1+(n)*m-m]=-kf/df/delta_r[n-1]/delta_r[n-1]
                        M[j-1+(n)*m,j-1+(n)*m+m]=-kf/df/delta_r[n-1]/delta_r[n-1]
                        y[j-1+(n)*m]=cf/delta_t*T_hu[j,n]

                    elif j == Fra[fra_num-2]:
                        A1=cf/delta_t-(-cf*VV[j,n])/delta_r[n-1]+2*kf/df/delta_r[n-1]/delta_r[n-1]+h/df/width+h/df/width
                        M[j-1+(n)*m,j-1+(n)*m]=A1
                        M[j-1+(n)*m,j+(n)*m]=-h/df/width
                        M[j-1+(n)*m,j-2+(n)*m]=-h/df/width
                        M[j-1+(n)*m,j-1+(n)*m-m]=(-cf*VV[j,n])/delta_r[n-1]-kf/df/delta_r[n-1]/delta_r[n-1]
                        M[j-1+(n)*m,j-1+(n)*m+m]=-kf/df/delta_r[n-1]/delta_r[n-1]
                        y[j-1+(n)*m]=cf/delta_t*T_hu[j,n]

                    elif j == Fra[fra_num-1]:
                        A1=cf/delta_t-(cf*VV[j,n])/delta_r[n-1]+2*kf/df/delta_r[n-1]/delta_r[n-1]+h/df/width+h/df/width
                        M[j-1+(n)*m,j-1+(n)*m]=A1
                        M[j-1+(n)*m,j+(n)*m]=-h/df/width
                        M[j-1+(n)*m,j-2+(n)*m]=-h/df/width
                        M[j-1+(n)*m,j-1+(n)*m-m]=-kf/df/delta_r[n-1]/delta_r[n-1]
                        M[j-1+(n)*m,j-1+(n)*m+m]=-kf/df/delta_r[n-1]/delta_r[n-1]+(cf*VV[j,n])/delta_r[n-1]
                        y[j-1+(n)*m]=cf/delta_t*T_hu[j,n]

                    elif j+1 in Fra[:fra_num-2]:#j=Fra-1
                        B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j-1]+kf/ke*1/delta_z[j])+2*alpha/delta_r[n-1]/delta_r[n-1]
                        M[j-1+(n)*m,j-1+(n)*m]=B1
                        M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]
                        M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*kf/ke*1/delta_z[j]
                        M[j-1+(n)*m,j-1+m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                        M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                        y[j-1+(n)*m]=1/delta_t*T_hu[j,n]

                    elif j-1 in Fra[:fra_num-2]:#j=Fra+1
                        B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j]+kf/ke*1/delta_z[j-1])+2*alpha/delta_r[n-1]/delta_r[n-1]
                        M[j-1+(n)*m,j-1+(n)*m]=B1
                        M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
                        M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*kf/ke*1/delta_z[j-1]
                        M[j-1+(n)*m,j-1+m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                        M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                        y[j-1+(n)*m]=1/delta_t*T_hu[j,n]

                    elif j+1 == Fra[fra_num-2]:#j=Fra-1
                        B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j-1]+h/ke)+2*alpha/delta_r[n-1]/delta_r[n-1]
                        M[j-1+(n)*m,j-1+(n)*m]=B1
                        M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]
                        M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*h/ke
                        M[j-1+(n)*m,j-1+m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                        M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                        y[j-1+(n)*m]=1/delta_t*T_hu[j,n]

                    elif j-1 == Fra[fra_num-2]:#j=Fra+1
                        B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j]+h/ke)+2*alpha/delta_r[n-1]/delta_r[n-1]
                        M[j-1+(n)*m,j-1+(n)*m]=B1
                        M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
                        M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*h/ke
                        M[j-1+(n)*m,j-1+m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                        M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                        y[j-1+(n)*m]=1/delta_t*T_hu[j,n]

                    elif j+1 == Fra[fra_num-1]:#j=Fra-1
                        B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j-1]+h/ke)+2*alpha/delta_r[n-1]/delta_r[n-1]
                        M[j-1+(n)*m,j-1+(n)*m]=B1
                        M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]
                        M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*h/ke
                        M[j-1+(n)*m,j-1+m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                        M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                        y[j-1+(n)*m]=1/delta_t*T_hu[j,n]

                    elif j-1 == Fra[fra_num-1]:#j=Fra+1
                        B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j]+h/ke)+2*alpha/delta_r[n-1]/delta_r[n-1]
                        M[j-1+(n)*m,j-1+(n)*m]=B1
                        M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
                        M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*h/ke
                        M[j-1+(n)*m,j-1+m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                        M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                        y[j-1+(n)*m]=1/delta_t*T_hu[j,n]    

                    else:
                        B=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j]+1/delta_z[j-1])+2*alpha/delta_r[n-1]/delta_r[n-1]
                        M[j-1+(n)*m,j-1+(n)*m]=B
                        M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]
                        M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
                        M[j-1+(n)*m,j-1+m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                        M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                        y[j-1+(n)*m]=1/delta_t*T_hu[j,n]
              #j=m
                B=1/delta_t+2*alpha/(delta_z[m]+delta_z[m-1])*(1/delta_z[m]+1/delta_z[m-1])+2*alpha/delta_r[n-1]/delta_r[n-1]
                M[m-1+(n)*m,m-1+(n)*m]=B
                M[m-1+(n)*m,m-2+(n)*m]=-2*alpha/(delta_z[m]+delta_z[m-1])*1/delta_z[m-1]
                M[m-1+(n)*m,m-1+m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                M[m-1+(n)*m,m-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                y[m-1+(n)*m]=1/delta_t*T_hu[m,n]+2*alpha/(delta_z[m]+delta_z[m-1])*1/delta_z[m]*(a*L+b)
            #n=N+1
            for n in range(NN+1,NN+2):
                for j in range(1,2):
                    B=1/delta_t+2*alpha/delta_z[j]/delta_z[j-1]+2*alpha/delta_r[n-1]/delta_r[n-1]
                    M[0+(n)*m,0+(n)*m]=B
                    M[0+(n)*m,1+(n)*m]=-(2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j])
                    M[0+(n)*m,-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                    y[0+(n)*m]=1/delta_t*T_hu[1,n]+(2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j-1])*T_hu[0,n]+alpha/delta_r[n-1]/delta_r[n-1]*(a*z[j]+b)

                for j in range(2,m):
                    if j in Fra[:fra_num-2]:
                        j=int(j)
                        A1=cf/delta_t+2*kf/df/delta_r[n-1]/delta_r[n-1]+kf/df/width/(width/2)+kf/df/width/(width/2)
                        M[j-1+(n)*m,j-1+(n)*m]=A1
                        M[j-1+(n)*m,j+(n)*m]=-kf/df/width/(width/2)
                        M[j-1+(n)*m,j-2+(n)*m]=-kf/df/width/(width/2)
                        M[j-1+(n)*m,j-1+(n)*m-m]=-kf/df/delta_r[n-1]/delta_r[n-1]
                        y[j-1+(n)*m]=cf/delta_t*T_hu[j,n]+kf/df/delta_r[n-1]/delta_r[n-1]*(a*z[j]+b)
                    elif j == Fra[fra_num-2]:
                        j=int(j)
                        A1=cf/delta_t-(-cf*VV[j,n])/delta_r[n-1]+2*kf/df/delta_r[n-1]/delta_r[n-1]+h/df/width+h/df/width
                        M[j-1+(n)*m,j-1+(n)*m]=A1
                        M[j-1+(n)*m,j+(n)*m]=-h/df/width
                        M[j-1+(n)*m,j-2+(n)*m]=-h/df/width
                        M[j-1+(n)*m,j-1+(n)*m-m]=(-cf*VV[j,n])/delta_r[n-1]-kf/df/delta_r[n-1]/delta_r[n-1]
                        y[j-1+(n)*m]=cf/delta_t*T_hu[j,n]+kf/df/delta_r[n-1]/delta_r[n-1]*(a*z[j]+b) 
                    elif j == Fra[fra_num-1]:
                        j=int(j)
                        A1=cf/delta_t-(cf*VV[j,n])/delta_r[n-1]+2*kf/df/delta_r[n-1]/delta_r[n-1]+h/df/width+h/df/width
                        M[j-1+(n)*m,j-1+(n)*m]=A1
                        M[j-1+(n)*m,j+(n)*m]=-h/df/width
                        M[j-1+(n)*m,j-2+(n)*m]=-h/df/width
                        M[j-1+(n)*m,j-1+(n)*m-m]=-kf/df/delta_r[n-1]/delta_r[n-1]
                        y[j-1+(n)*m]=cf/delta_t*T_hu[j,n]+(kf/df/delta_r[n-1]/delta_r[n-1]-(cf*VV[j,n])/delta_r[n-1])*(a*z[j]+b)
                    elif j+1 in Fra[:fra_num-2]:#j=Fra-1
                        B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j-1]+kf/ke*1/delta_z[j])+2*alpha/delta_r[n-1]/delta_r[n-1]
                        M[j-1+(n)*m,j-1+(n)*m]=B1
                        M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]
                        M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*kf/ke*1/delta_z[j]
                        M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                        y[j-1+(n)*m]=1/delta_t*T_hu[j,n]+alpha/delta_r[n-1]/delta_r[n-1]*(a*z[j]+b)

                    elif j-1 in Fra[:fra_num-2]:
                        B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j]+kf/ke*1/delta_z[j-1])+2*alpha/delta_r[n-1]/delta_r[n-1]
                        M[j-1+(n)*m,j-1+(n)*m]=B1
                        M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
                        M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*kf/ke*1/delta_z[j-1]
                        M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                        y[j-1+(n)*m]=1/delta_t*T_hu[j,n]+alpha/delta_r[n-1]/delta_r[n-1]*(a*z[j]+b)
                    elif j+1 == Fra[fra_num-2]:#j=Fra-1
                        B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j-1]+h/ke)+2*alpha/delta_r[n-1]/delta_r[n-1]
                        M[j-1+(n)*m,j-1+(n)*m]=B1
                        M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]
                        M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*h/ke
                        M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                        y[j-1+(n)*m]=1/delta_t*T_hu[j,n]+alpha/delta_r[n-1]/delta_r[n-1]*(a*z[j]+b)

                    elif j-1 == Fra[fra_num-2]:
                        B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j]+h/ke)+2*alpha/delta_r[n-1]/delta_r[n-1]
                        M[j-1+(n)*m,j-1+(n)*m]=B1
                        M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
                        M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*h/ke
                        M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                        y[j-1+(n)*m]=1/delta_t*T_hu[j,n]+alpha/delta_r[n-1]/delta_r[n-1]*(a*z[j]+b)
                    elif j+1 == Fra[fra_num-1]:#j=Fra-1
                        B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j-1]+h/ke)+2*alpha/delta_r[n-1]/delta_r[n-1]
                        M[j-1+(n)*m,j-1+(n)*m]=B1
                        M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]
                        M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*h/ke
                        M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                        y[j-1+(n)*m]=1/delta_t*T_hu[j,n]+alpha/delta_r[n-1]/delta_r[n-1]*(a*z[j]+b)

                    elif j-1 == Fra[fra_num-1]:
                        B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j]+h/ke)+2*alpha/delta_r[n-1]/delta_r[n-1]
                        M[j-1+(n)*m,j-1+(n)*m]=B1
                        M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
                        M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*h/ke
                        M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                        y[j-1+(n)*m]=1/delta_t*T_hu[j,n]+alpha/delta_r[n-1]/delta_r[n-1]*(a*z[j]+b)

                    else:
                        B=1/delta_t+2*alpha/delta_z[j]/delta_z[j-1]+2*alpha/delta_r[n-1]/delta_r[n-1]
                        M[j-1+(n)*m,j-1+(n)*m]=B
                        M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]
                        M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
                        M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                        y[j-1+(n)*m]=1/delta_t*T_hu[j,n]+alpha/delta_r[n-1]/delta_r[n-1]*(a*z[j]+b)
              #j=m
                B=1/delta_t+2*alpha/delta_z[m]/delta_z[m-1]+2*alpha/delta_r[n-1]/delta_r[n-1]
                M[m-1+(n)*m,m-1+(n)*m]=B
                M[m-1+(n)*m,m-2+(n)*m]=-2*alpha/(delta_z[m]+delta_z[m-1])*1/delta_z[m-1]
                M[m-1+(n)*m,m-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                y[m-1+(n)*m]=1/delta_t*T_hu[m,n]+2*alpha/(delta_z[m]+delta_z[m-1])*1/delta_z[m]*(a*L+b)+alpha/delta_r[n-1]/delta_r[n-1]*(a*z[j]+b)
            y_tf=np.expand_dims(y,axis=1)
            M_gpu=tf.convert_to_tensor(M)
            y_gpu=tf.convert_to_tensor(y_tf)
            with tf.device('/GPU:0'):
                T_new_f=tf.linalg.solve(M_gpu,y_gpu)
            T_new_f.numpy()
            for n in range(0,NN+2):
                for j in range(1,m+1):
                    k=j+(n)*m-1
                    T_hu[j,n]=T_new_f[k]#formation更新温度
        T_heatup_output[tt,:,ii]=T_hu[:,0]
        Z_train_out[tt,:,ii]=z
        print(f"tt={tt}")
        T_frac_r[ii,:]=T_hu[int(Fra[-1]),:]
df11=pd.DataFrame(T_heatup_output.reshape(Num_train*(m+1),heat_up_number))
df11.to_csv('2fracture_T_heatup_multistage_New_type_geo_initial_for_0229.csv',index=None)#wellbore temperature
dfz=pd.DataFrame(Z_train_out.reshape(Num_train*(m+1),heat_up_number))
dfz.to_csv('2fracture_multistage_realposition_New_type_geo_initial_for_0229.csv',index=None)#real depth

In [ ]:

#warmback temperature generation
r1=0.1
ke=2
cf=4100
kf=0.5
alpha=1.0e-6 
df=1000
a=0.2
b=21
Tinj=10
R=200 
width=0.1
L=900
m=900
NN=20 
delta_t=3600*2
heat_up_number=10
Num_train=800
T_heatup_output=np.zeros((Num_train,m+1,heat_up_number))
Z_train_out=np.zeros((Num_train,m+1,heat_up_number))
T_heatup_train=np.zeros((Num_train,m+1,NN+2))
y_original=np.zeros((Num_train,m+1))
Z_train=np.zeros((Num_train,m+1))
Z_train=np.zeros((Num_train,m+1))
T_injection= np.array(pd.read_csv('T_injection_train_multifrac_iso.csv'))#load temperature at the end of injection as initial field 
T_injection_train=T_injection.reshape(Num_train,m+1,NN+2)
L_p=np.array(pd.read_csv('fracture_L1_multifrac_iso.csv'))#load the same fracture distribution with that generated in injection cases
#logarithmic cylindrical coordinates:
r=np.zeros(NN+1)
delta_r=np.zeros(NN+1)
delta_rr=1/NN*math.log(R/r1)
for n in range(1,NN+2):
  r[n-1]=r1*math.pow(R/r1,(n-1)/NN)
  delta_r[n-1]=delta_rr*r[n-1] 
for tt in range(0,Num_train):#number of training data 
      T_hu=np.zeros((m+1,NN+2))
      T_hu=T_injection_train[tt,:,:]
      L1=[]
      for k in range(0,m):
          if L_p[tt,k]==1:
              L1.append(k)
      fra_num=len(L1)
      Fra=np.zeros(fra_num)
      for i in range(0,fra_num):
          Fra[i]=int(m*L1[i]/L)      
      z=np.zeros(m+1)
      delta_z=np.zeros(m+1)
      for i in range(0,fra_num-1):
        for j in range(int(Fra[i]+1),int(Fra[i+1]-1)):
          if ((Fra[i+1]-1)-(Fra[i]+1))%2==0:
            N=int(((Fra[i+1]-1)-(Fra[i]+1))/2) 
            z1=np.zeros(N+1)
            delta_z1=np.zeros(N)
            for k in range(0,N+1):
              z1[k]=1/(1/(L1[i]+width/2)-k/N*(1/(L1[i]+width/2)-1/((L1[i]+L1[i+1])/2)))
            for k in range(0,N):
              delta_z1[k]=z1[k+1]-z1[k]
          elif ((Fra[i+1]-1)-(Fra[i]+1)+1)%2==0:
            N=int(((Fra[i+1]-1)-(Fra[i]+1)+1)/2)
            z1=np.zeros(N+1)
            delta_z1=np.zeros(N)
            L3=((L1[i+1]-width/2)-(2*N-2)*(L1[i]+width/2)+(4*(L1[i]+width/2)**2*(N**2-N)+(L1[i+1]-width/2)**2)**0.5)/2
            for k in range(0,N+1):
              z1[k]=1/(1/(L1[i]+width/2)-k/N*(1/(L1[i]+width/2)-1/L3))
            for k in range(0,N):
              delta_z1[k]=z1[k+1]-z1[k]
          delta_z12=np.zeros(int(Fra[i+1]-1-(Fra[i]+1)))
          for n in range(0,int(Fra[i+1]-1-(Fra[i]+1))):
            if n<N:
              delta_z12[n]=delta_z1[n]
            elif n>=N:
              delta_z12[n]=delta_z1[int(Fra[i+1]-1-(Fra[i]+1)-n-1)]
          delta_z[j]=delta_z12[int(j-(Fra[i]+1))]
        delta_z[int(Fra[i+1]-1)]=width/2
        delta_z[int(Fra[i])]=width/2
      z1=np.zeros(int(Fra[0]))
      delta_z1=np.zeros(int(Fra[0]-1))
      for k in range(0,int(Fra[0])):
        z1[k]=width/2*math.pow(L1[0]/(width/2),k/(Fra[0]-1))
      for k in range(0,int(Fra[0]-1)):
        delta_z1[k]=z1[k+1]-z1[k]
      for j in range(0,int(Fra[0]-1)):
        delta_z[j]=delta_z1[int(Fra[0]-2-j)]
      delta_z[int(Fra[0]-1)]=width/2
      z2=np.zeros(int(m-Fra[fra_num-1]))
      delta_z2=np.zeros(int(m-Fra[fra_num-1]-1))
      for k in range(0,int(m-Fra[fra_num-1])):
        z2[k]=width/2*math.pow((L-L1[fra_num-1])/(width/2),k/(m-Fra[fra_num-1]-1))
      for k in range(0,int(m-Fra[fra_num-1]-1)):
        delta_z2[k]=z2[k+1]-z2[k]
      for j in range(int(Fra[fra_num-1]+1),m):
        delta_z[j]=delta_z2[int(j-Fra[fra_num-1]-1)]
      delta_z[int(Fra[fra_num-1])]=width/2
      delta_z[m]=delta_z[m-1]
      z[0]=0
      for j in range(1,m+1):
        z[j]=sum(delta_z[0:j])
      # initial condition
      T_new_f=np.zeros((m+1,NN+2))
      y=np.zeros(m*(NN+2))
      M=np.zeros((m*(NN+2),m*(NN+2)))
      # boundary condition
      T_hu[0,0]=b 
      T_hu[0,1:]=b 
      for ii in range(0,heat_up_number):
          for t in range(0,1,1):
            #wellbore
            for j in range(1,2):
              D=cf/delta_t+2/r1/df*kf/(r1/2)+2*kf/df/(delta_z[j]+delta_z[j-1])*(1/delta_z[j-1]+1/delta_z[j])
              M[0,0]=D
              M[0,1]=-2*kf/df/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
              M[0,m]=-2/r1/df*kf/(r1/2)
              y[0]=cf/delta_t*T_hu[j,0]+2*kf/df/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]*T_hu[0,0]
            for j in range(2,m):
              if j in Fra:
                D=cf/delta_t+2*kf/df/(width/2)/(width/2)
                M[j-1,j-1]=D
                M[j-1,j-2]=-kf/df/(width/2)/(width/2)
                M[j-1,j]=-kf/df/(width/2)/(width/2)
                M[j-1,j-1+m]=0
                y[j-1]=cf/delta_t*T_hu[j,0]
              else: #j!=Fra
                D=cf/delta_t+2/r1/df*kf/(r1/2)+2*kf/df/(delta_z[j]+delta_z[j-1])*(1/delta_z[j-1]+1/delta_z[j])
                M[j-1,j-1]=D
                M[j-1,j]=-2*kf/df/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
                M[j-1,j-2]=-2*kf/df/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]
                M[j-1,j-1+m]=-2/r1/df*kf/(r1/2)
                y[j-1]=cf/delta_t*T_hu[j,0]
            #j=m
            D=cf/delta_t+2/r1/df*kf/(r1/2)+2*kf/df/(delta_z[m]+delta_z[m-1])*(1/delta_z[m-1]+1/delta_z[m])
            M[m-1,m-1]=D
            M[m-1,m-2]=-2*kf/df/(delta_z[m]+delta_z[m-1])*1/delta_z[m-1]
            #M[m-1,m]=-2*kf/df/(delta_z[m]+delta_z[m-1])*1/delta_z[m]
            M[m-1,m-1+m]=-2/r1/df*kf/(r1/2)
            y[m-1]=cf/delta_t*T_hu[j,0]+2*kf/df/(delta_z[m]+delta_z[m-1])*1/delta_z[m]*(a*L+b)
            #formation
            #n=1
            for n in range(1,2):
              for j in range(1,2):
                B=1/delta_t+2*alpha/delta_z[j]/delta_z[j-1]+alpha/delta_r[1-1]/delta_r[1-1]+kf/ke*alpha/delta_r[1-1]/(r1/2)
                M[0+(n)*m,0+(n)*m]=B
                M[0+(n)*m,1+(n)*m]=-(2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j])
                M[0+m,m+m]=-(alpha/delta_r[1-1]/delta_r[1-1])
                M[0+m,m-m]=-kf/ke*alpha/delta_r[1-1]/(r1/2)
                y[0+(n)*m]=1/delta_t*T_hu[1,n]+(2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j-1])*T_hu[0,n]
              for j in range(2,m):
                if j in Fra:
                  M[j-1+m,j-1+m]=1
                  M[j-1+m,j-1+m-m]=-1
                  y[j-1+m]=0 
                elif j+1 in Fra:#j==Fra-1
                  B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j-1]+kf/ke*1/delta_z[j])+alpha/delta_r[1-1]/delta_r[1-1]+kf/ke*alpha/delta_r[1-1]/(r1/2)
                  M[j-1+(n)*m,j-1+(n)*m]=B1
                  M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]
                  M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*kf/ke*1/delta_z[j]
                  M[j-1+m,j-1+m+m]=-(alpha/delta_r[1-1]/delta_r[1-1])
                  M[j-1+m,j-1+m-m]=-kf/ke*alpha/delta_r[1-1]/(r1/2)
                  y[j-1+(n)*m]=1/delta_t*T_hu[j,n]

                elif j-1 in Fra:#j==Fra+1
                  B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j]+kf/ke*1/delta_z[j-1])+alpha/delta_r[1-1]/delta_r[1-1]+kf/ke*alpha/delta_r[1-1]/(r1/2)
                  M[j-1+(n)*m,j-1+(n)*m]=B1
                  M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
                  M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*kf/ke*1/delta_z[j-1]
                  M[j-1+m,j-1+m+m]=-(alpha/delta_r[1-1]/delta_r[1-1])
                  M[j-1+m,j-1+m-m]=-kf/ke*alpha/delta_r[1-1]/(r1/2)
                  y[j-1+(n)*m]=1/delta_t*T_hu[j,n]

                else:
                    B=1/delta_t+2*alpha/delta_z[j]/delta_z[j-1]+alpha/delta_r[1-1]/delta_r[1-1]+kf/ke*alpha/delta_r[1-1]/(r1/2)
                    M[j-1+(n)*m,j-1+(n)*m]=B
                    M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]
                    M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
                    M[j-1+m,j-1+m+m]=-(alpha/delta_r[1-1]/delta_r[1-1])
                    M[j-1+m,j-1+m-m]=-kf/ke*alpha/delta_r[1-1]/(r1/2)
                    y[j-1+(n)*m]=1/delta_t*T_hu[j,n]
              #j=m
              B=1/delta_t+2*alpha/delta_z[m]/delta_z[m-1]+alpha/delta_r[1-1]/delta_r[1-1]+kf/ke*alpha/delta_r[1-1]/(r1/2)
              M[m-1+(n)*m,m-1+(n)*m]=B
              M[m-1+(n)*m,m-2+(n)*m]=-2*alpha/(delta_z[m]+delta_z[m-1])*1/delta_z[m-1]
              M[m-1+m,m-1+m+m]=-(alpha/delta_r[1-1]/delta_r[1-1])
              M[m-1+m,m-1+m-m]=-kf/ke*alpha/delta_r[1-1]/(r1/2)
              y[m-1+(n)*m]=1/delta_t*T_hu[m,n]+2*alpha/(delta_z[m]+delta_z[m-1])*1/delta_z[m]*(a*L+b)
            #n=2...N
            for n in range(2,NN+1):
              for j in range(1,2):
                B=1/delta_t+2*alpha/delta_z[j]/delta_z[j-1]+2*alpha/delta_r[n-1]/delta_r[n-1]
                M[0+(n)*m,0+(n)*m]=B
                M[0+(n)*m,1+(n)*m]=-(2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j])
                M[0+(n)*m,m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                M[0+(n)*m,-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                y[0+(n)*m]=1/delta_t*T_hu[1,n]+(2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j-1])*T_hu[0,n]

              for j in range(2,m):
                if j in Fra:
                  A1=cf/delta_t+2*kf/df/delta_r[n-1]/delta_r[n-1]+kf/df/width/(width/2)+kf/df/width/(width/2)
                  M[j-1+(n)*m,j-1+(n)*m]=A1
                  M[j-1+(n)*m,j+(n)*m]=-kf/df/width/(width/2)
                  M[j-1+(n)*m,j-2+(n)*m]=-kf/df/width/(width/2)
                  M[j-1+(n)*m,j-1+(n)*m-m]=-kf/df/delta_r[n-1]/delta_r[n-1]
                  M[j-1+(n)*m,j-1+(n)*m+m]=-kf/df/delta_r[n-1]/delta_r[n-1]
                  y[j-1+(n)*m]=cf/delta_t*T_hu[j,n]

                elif j+1 in Fra:#j=Fra-1
                  B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j-1]+kf/ke*1/delta_z[j])+2*alpha/delta_r[n-1]/delta_r[n-1]
                  M[j-1+(n)*m,j-1+(n)*m]=B1
                  M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]
                  M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*kf/ke*1/delta_z[j]
                  M[j-1+(n)*m,j-1+m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                  M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                  y[j-1+(n)*m]=1/delta_t*T_hu[j,n]

                elif j-1 in Fra:#j=Fra+1
                  B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j]+kf/ke*1/delta_z[j-1])+2*alpha/delta_r[n-1]/delta_r[n-1]
                  M[j-1+(n)*m,j-1+(n)*m]=B1
                  M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
                  M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*kf/ke*1/delta_z[j-1]
                  M[j-1+(n)*m,j-1+m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                  M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                  y[j-1+(n)*m]=1/delta_t*T_hu[j,n]
                else:
                    B=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j]+1/delta_z[j-1])+2*alpha/delta_r[n-1]/delta_r[n-1]
                    M[j-1+(n)*m,j-1+(n)*m]=B
                    M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]
                    M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
                    M[j-1+(n)*m,j-1+m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                    M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                    y[j-1+(n)*m]=1/delta_t*T_hu[j,n]
              #j=m
              B=1/delta_t+2*alpha/(delta_z[m]+delta_z[m-1])*(1/delta_z[m]+1/delta_z[m-1])+2*alpha/delta_r[n-1]/delta_r[n-1]
              M[m-1+(n)*m,m-1+(n)*m]=B
              M[m-1+(n)*m,m-2+(n)*m]=-2*alpha/(delta_z[m]+delta_z[m-1])*1/delta_z[m-1]
              M[m-1+(n)*m,m-1+m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
              M[m-1+(n)*m,m-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
              y[m-1+(n)*m]=1/delta_t*T_hu[m,n]+2*alpha/(delta_z[m]+delta_z[m-1])*1/delta_z[m]*(a*L+b)
            #n=N+1
            for n in range(NN+1,NN+2):
              for j in range(1,2):
                B=1/delta_t+2*alpha/delta_z[j]/delta_z[j-1]+2*alpha/delta_r[n-1]/delta_r[n-1]
                M[0+(n)*m,0+(n)*m]=B
                M[0+(n)*m,1+(n)*m]=-(2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j])
                M[0+(n)*m,-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                y[0+(n)*m]=1/delta_t*T_hu[1,n]+(2*alpha/(delta_z[j]+delta_z[j-1])/delta_z[j-1])*T_hu[0,n]+alpha/delta_r[n-1]/delta_r[n-1]*(a*z[j]+b)

              for j in range(2,m):
                if j in Fra:
                  A1=cf/delta_t+2*kf/df/delta_r[n-1]/delta_r[n-1]+kf/df/width/(width/2)+kf/df/width/(width/2)
                  M[j-1+(n)*m,j-1+(n)*m]=A1
                  M[j-1+(n)*m,j+(n)*m]=-kf/df/width/(width/2)
                  M[j-1+(n)*m,j-2+(n)*m]=-kf/df/width/(width/2)
                  M[j-1+(n)*m,j-1+(n)*m-m]=-kf/df/delta_r[n-1]/delta_r[n-1]
                  y[j-1+(n)*m]=cf/delta_t*T_hu[j,n]+kf/df/delta_r[n-1]/delta_r[n-1]*(a*z[j]+b)

                elif j+1 in Fra:#j=Fra-1
                  B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j-1]+kf/ke*1/delta_z[j])+2*alpha/delta_r[n-1]/delta_r[n-1]
                  M[j-1+(n)*m,j-1+(n)*m]=B1
                  M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]
                  M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*kf/ke*1/delta_z[j]
                  M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                  y[j-1+(n)*m]=1/delta_t*T_hu[j,n]+alpha/delta_r[n-1]/delta_r[n-1]*(a*z[j]+b)

                elif j-1 in Fra:
                  B1=1/delta_t+2*alpha/(delta_z[j]+delta_z[j-1])*(1/delta_z[j]+kf/ke*1/delta_z[j-1])+2*alpha/delta_r[n-1]/delta_r[n-1]
                  M[j-1+(n)*m,j-1+(n)*m]=B1
                  M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
                  M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*kf/ke*1/delta_z[j-1]
                  M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                  y[j-1+(n)*m]=1/delta_t*T_hu[j,n]+alpha/delta_r[n-1]/delta_r[n-1]*(a*z[j]+b)
                else:
                    B=1/delta_t+2*alpha/delta_z[j]/delta_z[j-1]+2*alpha/delta_r[n-1]/delta_r[n-1]
                    M[j-1+(n)*m,j-1+(n)*m]=B
                    M[j-1+(n)*m,j-2+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j-1]
                    M[j-1+(n)*m,j+(n)*m]=-2*alpha/(delta_z[j]+delta_z[j-1])*1/delta_z[j]
                    M[j-1+(n)*m,j-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
                    y[j-1+(n)*m]=1/delta_t*T_hu[j,n]+alpha/delta_r[n-1]/delta_r[n-1]*(a*z[j]+b)
              #j=m
              B=1/delta_t+2*alpha/delta_z[m]/delta_z[m-1]+2*alpha/delta_r[n-1]/delta_r[n-1]
              M[m-1+(n)*m,m-1+(n)*m]=B
              M[m-1+(n)*m,m-2+(n)*m]=-2*alpha/(delta_z[m]+delta_z[m-1])*1/delta_z[m-1]
              M[m-1+(n)*m,m-1-m+(n)*m]=-alpha/delta_r[n-1]/delta_r[n-1]
              y[m-1+(n)*m]=1/delta_t*T_hu[m,n]+2*alpha/(delta_z[m]+delta_z[m-1])*1/delta_z[m]*(a*L+b)+alpha/delta_r[n-1]/delta_r[n-1]*(a*z[j]+b) 
            y_tf=np.expand_dims(y,axis=1)
            M_gpu=tf.convert_to_tensor(M)
            y_gpu=tf.convert_to_tensor(y_tf)
            with tf.device('/GPU:0'):
                T_new_f=tf.linalg.solve(M_gpu,y_gpu)
            T_new_f.numpy()
            for n in range(0,NN+2):
                for j in range(1,m+1):
                    k=j+(n)*m-1
                    T_hu[j,n]=T_new_f[k]
          T_heatup_output[tt,:,ii]=T_hu[:,0]
          Z_train_out[tt,:,ii]=z
          print(f"tt={tt}")
df11=pd.DataFrame(T_heatup_output.reshape(Num_train*(m+1),heat_up_number))
df11.to_csv('2fracture_T_heatup_multistage.csv',index=None)#multistage warmback wellbore temperature
dfz=pd.DataFrame(Z_train_out.reshape(Num_train*(m+1),heat_up_number))
dfz.to_csv('2fracture_multistage_realposition_0228.csv',index=None)

tt=0
tt=0
tt=0
tt=0
tt=0
tt=0
tt=0
tt=0
tt=0
tt=0
tt=1
tt=1
tt=1
tt=1
tt=1
tt=1
tt=1
tt=1
tt=1
tt=1
tt=2
tt=2
tt=2
tt=2
tt=2
tt=2
tt=2
tt=2
tt=2
tt=2
tt=3
tt=3
tt=3
tt=3
tt=3
tt=3
tt=3
tt=3
tt=3
tt=3
tt=4
tt=4
tt=4
tt=4
tt=4
tt=4
tt=4
tt=4
tt=4
tt=4
tt=5
tt=5
tt=5
tt=5
tt=5
tt=5
tt=5
tt=5
tt=5
tt=5
tt=6
tt=6
tt=6
tt=6
tt=6
tt=6
tt=6
tt=6
tt=6
tt=6
tt=7
tt=7
tt=7
tt=7
tt=7
tt=7
tt=7
tt=7
tt=7
tt=7
tt=8
tt=8
tt=8
tt=8
tt=8
tt=8
tt=8
tt=8
tt=8
tt=8
tt=9
tt=9
tt=9
tt=9
tt=9
tt=9
tt=9
tt=9
tt=9
tt=9
tt=10
tt=10
tt=10
tt=10
tt=10
tt=10
tt=10
tt=10
tt=10
tt=10
tt=11
tt=11
tt=11
tt=11
tt=11
tt=11
tt=11
tt=11
tt=11
tt=11
tt=12
tt=12
tt=12
tt=12
tt=12
tt=12
tt=12
tt=12
tt=12
tt=12
tt=13
tt=13
tt=13
tt=13
tt=13
tt=13
tt=13
tt=13
tt=13
tt=13
tt=14
tt=14
tt=14
tt=14
tt=14
tt=14
tt=14
tt=14
tt=14
tt=14
tt=15
tt=15
tt=15
tt=15
tt=15
tt=15
tt=15
tt=15
tt=15
tt=15
tt=16
tt=16
tt=16
tt=16
tt=16
tt=16
tt=16
tt=16
tt=16
tt=16
tt=17
tt=17
tt=17
tt=17
tt=17
tt=17
tt=17
tt=17
tt=17
tt=17
tt=18
tt=18
tt=18
tt